In [157]:
#!/home/alon/LocalInstalls/miniconda3/envs/pyoccenv741/bin/python
import ifcopenshell, ifcopenshell.geom

import sys
import os
sys.path.append(os.path.abspath('..'))
from src import ifcControl

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import art3d

import trimesh # https://github.com/mikedh/trimesh
import PIL
import io

In [13]:
# Open the IFC file using IfcOpenShell
ifc = ifcopenshell.open("../data/IfcOpenHouse_IFC4.ifc")
ifcControl.ifcshow(ifc)

INFO:OCC.Display.backend:The qt-pyqt5 backend is already loaded...``load_backend`` can only be called once per session
INFO:OCC.Display.SimpleGui:GUI backend set to: qt-pyqt5
 ###### 3D rendering pipe initialisation #####
Display3d class initialization starting ...
Aspect_DisplayConnection created.
Graphic_Driver created.
V3d_Viewer created.
AIS_InteractiveContext created.
V3d_View created
Xw_Window created.
Display3d class successfully initialized.
 ########################################


In [14]:
#https://sourceforge.net/p/ifcopenshell/discussion/1782716/thread/6e9f0aee79/
# https://framagit.org/pythonhvac/pythoncvc-net/blob/master/IfcOpenShellSamples/ifc_placement.py
# https://pythoncvc.net/?cat=183
# https://sourceforge.net/p/ifcopenshell/discussion/1782717/thread/409ef11620/

# Get a list of all walls in the file
products = ifc.by_type("IfcProduct")
settingsGeom = ifcopenshell.geom.settings()
settingsGeom.set(settingsGeom.USE_PYTHON_OPENCASCADE, False) #ifcopenshell.geom.create_shape behaives diffrently
settingsGeom.set(settingsGeom.USE_WORLD_COORDS,True)
settingsStyles = ifcopenshell.geom.settings()
settingsStyles.set(settingsStyles.USE_PYTHON_OPENCASCADE, True)
settingsStyles.set(settingsStyles.USE_WORLD_COORDS,True)

trigs = []
for product in products:
    if product.is_a("IfcOpeningElement"): continue
    if product.Representation:
        #collect color
        shapeStyle = ifcopenshell.geom.create_shape(settingsStyles, inst=product)
        color = np.array(shapeStyle.styles[0]) # the shape color

        #collect geometry
        shape = ifcopenshell.geom.create_shape(settingsGeom, inst=product)
        element = ifc.by_guid(shape.guid)
        verts = shape.geometry.verts # X Y Z of vertices in flattened list e.g. [v1x, v1y, v1z, v2x, v2y, v2z, ...]
        verts = np.array(verts).reshape((-1,3))
        faces = shape.geometry.faces  #Indices of vertices per triangle face e.g. [f1v1, f1v2, f1v3, f2v1, f2v2, f2v3, ...]
        faces = np.array(faces).reshape((-1,3))
        dict = {
            "element": element,
            "vertices": verts,
            "faces": faces,
            "color": color,
            }
        trigs.append(dict)

In [114]:
#https://stackoverflow.com/questions/56559379/plot-mesh-stored-as-vertices-and-faces

#-----dont want inline
%matplotlib auto 
#-----
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

for trig in trigs:
    v = trig["vertices"]
    f = trig["faces"]
    pc = art3d.Poly3DCollection(v[f], facecolors=trig["color"][:3], edgecolor="none")
    ax.add_collection(pc)

#auto scaling doesnt work
allV = np.concatenate([trigs[_]["vertices"] for _ in range(len(trigs))])
maxV = allV.max(axis=0)
minV = allV.min(axis=0)
ax.set(xlim=(minV[0], maxV[0]), ylim=(minV[1], maxV[1]), zlim=(minV[2], maxV[2]))
ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
plt.show()

DEBUG:matplotlib.pyplot:Loaded backend Qt5Agg version unknown.
Using matplotlib backend: Qt5Agg


In [16]:
pitch = 1
for trig in trigs:
    mesh = trimesh.Trimesh(vertices = trig["vertices"],faces = trig["faces"]) #type = trimesh.base.Trimesh
    voxels = mesh.voxelized(pitch)
    trig["mesh"] = mesh
    mesh.visual = trimesh.visual.ColorVisuals(mesh,face_colors=trig["color"])
    trig["voxel"] = voxels

DEBUG:trimesh:voxelize_subdivide executed in 0.0147 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0078 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0068 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0056 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0062 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0050 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0019 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0024 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0015 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0027 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0014 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0015 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0014 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0013 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0020 seconds.
DEBUG:trimesh:voxelize_subdivide executed in 0.0010 seconds.
DEBUG:trimesh:voxelize_s

In [17]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')

for tri in trigs:
    p = tri["voxel"].points
    ax.scatter(p[:,0],p[:,1],p[:,2],c=tri["color"][:3].reshape(1,-1))

In [122]:
#https://trimsh.org/examples/ray.html
ray_origins = np.array([0, -2.5, 2]) 
ray_directions = np.array([np.sin(0), np.cos(0), 0])

dDist = np.inf
loc = []
for trig in trigs:
    mesh = trig["mesh"]
    locations, index_ray, index_tri = mesh.ray.intersects_location([ray_origins],[ray_directions],multiple_hits = False)
    if len(locations>0):
        dVec = locations - ray_origins
        dDist_challanger = np.linalg.norm(dVec)
        if dDist > dDist_challanger:
            dDist = dDist_challanger
            loc = locations
print(dDist)
print(loc)

2.32
[[ 0.   -0.18  2.  ]]


In [162]:
#they have camera to rays module! nice :)
# https://github.com/mikedh/trimesh/issues/486
# https://github.com/mikedh/trimesh/issues/620
scene = trimesh.Scene()
for tri in trigs:
    scene.add_geometry(tri["mesh"])
scene.set_camera(resolution = (10,10),fov = (45,45))

DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimesh:1 items cleared from cache: ['nodes']
DEBUG:trimes

<trimesh.scene.Camera> FOV: [45. 45.] Resolution: [10 10]

In [163]:
# https://github.com/mikedh/trimesh/issues/1000
def ray_trace():
    # convert the camera to rays with one ray per pixel
    origins, vectors, pixels = scene.camera_rays()
    # do the actual ray- mesh queries
    points, index_ray, index_tri = mesh.ray.intersects_location(
        origins, vectors, multiple_hits=False)

    # for each hit, find the distance along its vector
    depth = trimesh.util.diagonal_dot(points - origins[0],
                                      vectors[index_ray])
    # find pixel locations of actual hits
    pixel_ray = pixels[index_ray]

    # create a numpy array we can turn into an image
    # doing it with uint8 creates an `L` mode greyscale image
    a = np.zeros(scene.camera.resolution, dtype=np.uint8)

    # scale depth against range (0.0 - 1.0)
    depth_float = ((depth - depth.min()) / depth.ptp())

    # convert depth into 0 - 255 uint8
    depth_int = (depth_float * 255).round().astype(np.uint8)
    # assign depth to correct pixel locations
    a += 255 
    a[pixel_ray[:, 0], pixel_ray[:, 1]] = depth_int
    # create a PIL image from the depth queries
    img = PIL.Image.fromarray(a)
    # show the resulting image
    display(img)

ray_trace()

DEBUG:trimesh:36 items cleared from cache: ['nodes_geometry', ('world', 'geometry_0'), ('world', 'geometry_1'), ('world', 'geometry_2'), ('world', 'geometry_3'), ('world', 'geometry_4'), ('world', 'geometry_5'), ('world', 'geometry_6'), ('world', 'geometry_7'), ('world', 'geometry_8'), ('world', 'geometry_9'), ('world', 'geometry_10'), ('world', 'geometry_11'), ('world', 'geometry_12'), ('world', 'geometry_13'), ('world', 'geometry_14'), ('world', 'geometry_15'), ('world', 'geometry_16'), ('world', 'geometry_17'), ('world', 'geometry_18'), ('world', 'geometry_19'), ('world', 'geometry_20'), ('world', 'geometry_21'), ('world', 'geometry_22'), ('world', 'geometry_23'), ('world', 'geometry_24'), ('world', 'geometry_25'), ('world', 'geometry_26'), ('world', 'geometry_27'), ('world', 'geometry_28'), ('world', 'geometry_29'), ('world', 'geometry_30'), ('world', 'geometry_31'), ('world', 'geometry_32'), ('world', 'geometry_33'), ('world', 'geometry_34')]


/tmp/ipykernel_1167/1978638841.py:20: RuntimeWarning: invalid value encountered in true_divide
  depth_float = ((depth - depth.min()) / depth.ptp())


In [ ]:
# https://colab.research.google.com/drive/1pcndwqeY8vker3bLKQNJKr3B-7-SYenE?usp=sharing#scrollTo=7uEYsxXjw9v0 
#more stuff